# 🌾 FarmFederate: Complete 17-Model Analysis with Paper Comparisons

## 🎯 Complete Analysis Pipeline:

### ✅ Models (17 Total):

#### 9 LLM Models (Text):
1. `google/flan-t5-small` - 60M params
2. `google/flan-t5-base` - 220M params
3. `t5-small` - 60M params
4. `gpt2` - 124M params
5. `gpt2-medium` - 355M params
6. `distilgpt2` - 82M params
7. `roberta-base` - 125M params
8. `bert-base-uncased` - 110M params
9. `distilbert-base-uncased` - 66M params

#### 4 ViT Models (Images):
1. `google/vit-base-patch16-224` - 86M params
2. `google/vit-large-patch16-224` - 304M params
3. `google/vit-base-patch16-384` - 86M params
4. `facebook/deit-base-patch16-224` - 86M params

#### 4 VLM Models (Text + Images):
1. `openai/clip-vit-base-patch32` - 151M params
2. `openai/clip-vit-large-patch14` - 428M params
3. `Salesforce/blip-image-captioning-base` - 385M params
4. `Salesforce/blip2-opt-2.7b` - 2.7B params

### ✅ Training Modes:

#### Federated Learning (Privacy-Preserving):
- 5 clients, 10 rounds
- Non-IID data split (Dirichlet α=0.5)
- FedAvg aggregation
- Communication efficiency tracking

#### Centralized Learning (Baseline):
- All data at server
- 10 epochs
- Standard training

### ✅ Datasets (4+ Sources Each):

#### Text:
1. CGIAR GARDIAN - Agricultural research
2. Argilla Farming - Q&A dataset
3. AG News - News articles
4. Agricultural QA - Question answering
5. LocalMini - Sensor logs (fallback)

#### Images:
1. PlantVillage - 54K+ disease images
2. Bangladesh Crop - 6K crop diseases
3. PlantWild - 6K wild plants
4. Plant Pathology 2021 - Kaggle dataset
5. Synthetic - Generated (fallback)

### ✅ Outputs:
- **9 comprehensive comparison plots**
- **Paper comparison analysis**
- **Communication efficiency metrics**
- **Complete benchmarking report**

### ✅ Paper Comparisons:
1. **McMahan et al. (2017)** - FedAvg baseline
2. **Li et al. (2020)** - FedProx
3. **Karimireddy et al. (2020)** - SCAFFOLD
4. **Chen et al. (2020)** - AgriNet (Agriculture AI)
5. **Singh et al. (2020)** - PlantDoc

---

## ⚙️ Step 1: Enable GPU (MANDATORY)

**Runtime → Change runtime type → GPU (A100 recommended) → Save**

In [ ]:
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ NO GPU! Enable GPU: Runtime → Change runtime type → GPU")

## 📦 Step 2: Install Dependencies & Clone Repository

In [ ]:
!pip install -q transformers>=4.40 datasets peft torch torchvision scikit-learn seaborn matplotlib numpy pandas pillow requests tqdm
print("✅ Dependencies installed!")

In [ ]:
!git clone -b feature/multimodal-work https://github.com/Solventerritory/FarmFederate-Advisor.git
%cd FarmFederate-Advisor/backend
!pwd
print("\n✅ Repository cloned!")

## 🔧 Step 3: Imports & Configuration

In [ ]:
import os
import gc
import time
import json
import random
import warnings
from typing import List, Dict, Tuple, Optional
from copy import deepcopy
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from PIL import Image
import torchvision.transforms as T

from transformers import (
    AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM,
    ViTModel, ViTImageProcessor,
    CLIPProcessor, CLIPModel,
    BlipProcessor, BlipForConditionalGeneration,
    Blip2Processor, Blip2ForConditionalGeneration,
    get_linear_schedule_with_warmup,
    logging as hf_logging
)

try:
    from peft import LoraConfig, get_peft_model
    HAS_PEFT = True
except:
    HAS_PEFT = False

# Import real dataset loaders
from datasets_loader import (
    build_text_corpus_mix,
    load_stress_image_datasets_hf,
    ISSUE_LABELS,
    NUM_LABELS
)

warnings.filterwarnings('ignore')
hf_logging.set_verbosity_error()

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n🚀 Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

print(f"\n📊 Plant Stress Labels ({NUM_LABELS}):")
for i, label in enumerate(ISSUE_LABELS):
    print(f"   {i}: {label}")

## 🔧 Step 4: Fixed LoRA Target Module Detection (All 17 Models)

In [ ]:
def get_lora_target_modules(model_name: str):
    """Auto-detect correct LoRA target modules for ALL 17 model architectures."""
    model_name_lower = model_name.lower()
    
    # T5 family (Flan-T5, T5)
    if "t5" in model_name_lower or "flan" in model_name_lower:
        return ["q", "v"]
    
    # BERT family (BERT, RoBERTa, DistilBERT, ALBERT)
    elif "bert" in model_name_lower or "roberta" in model_name_lower or "albert" in model_name_lower:
        return ["query", "value"]
    
    # GPT family (GPT-2, DistilGPT2)
    elif "gpt" in model_name_lower:
        return ["c_attn"]
    
    # Vision Transformers (ViT, DeiT, Swin)
    elif "vit" in model_name_lower or "deit" in model_name_lower or "swin" in model_name_lower:
        return ["query", "value"]
    
    # CLIP
    elif "clip" in model_name_lower:
        return ["q_proj", "v_proj"]
    
    # BLIP family
    elif "blip" in model_name_lower:
        return ["query", "value"]
    
    # Safe default
    else:
        return ["query", "value"]

print("✅ LoRA target module detection loaded (supports all 17 models)")

# Test all model families
test_models = [
    "google/flan-t5-small", "gpt2", "bert-base-uncased", 
    "google/vit-base-patch16-224", "openai/clip-vit-base-patch32", 
    "Salesforce/blip-image-captioning-base"
]
print("\n📋 Target modules for each family:")
for model in test_models:
    modules = get_lora_target_modules(model)
    print(f"   {model.split('/')[-1]:30s} → {modules}")

## 📊 Step 5: Load MULTIPLE Real Datasets (4+ Sources Each)

In [ ]:
print("\n" + "="*70)
print("LOADING MULTIPLE TEXT DATASETS (4+ REAL SOURCES)")
print("="*70)

# Load text datasets: CGIAR GARDIAN, Argilla Farming, AG News, Agricultural QA, LocalMini
text_df = build_text_corpus_mix(
    mix_sources="gardian,argilla,agnews,localmini",
    max_per_source=1000,
    max_samples=5000
)

print(f"\n✅ Total text samples loaded: {len(text_df)}")

# Show dataset breakdown
if 'source' in text_df.columns:
    print("\n📊 Text dataset source breakdown:")
    source_counts = text_df['source'].value_counts()
    for source, count in source_counts.items():
        print(f"   {source:15s}: {count:4d} samples")
    text_sources = text_df['source'].tolist()
else:
    text_sources = ['mixed'] * len(text_df)

text_data = text_df['text'].tolist()
text_labels = text_df['labels'].tolist()

# Label distribution
print("\n📊 Text label distribution:")
label_counts = np.zeros(NUM_LABELS)
for labels in text_labels:
    for label_idx in labels:
        label_counts[label_idx] += 1
for i, count in enumerate(label_counts):
    print(f"   {ISSUE_LABELS[i]:15s}: {int(count):4d} samples")

In [ ]:
print("\n" + "="*70)
print("LOADING MULTIPLE IMAGE DATASETS (4+ REAL SOURCES)")
print("="*70)

# Load image datasets: PlantVillage, Bangladesh, PlantWild, Plant Pathology 2021
image_dataset_hf = load_stress_image_datasets_hf(
    max_total_images=6000,
    max_per_dataset=2000
)

if image_dataset_hf is not None:
    print(f"\n✅ Total real images loaded: {len(image_dataset_hf)}")
    
    image_data = []
    image_labels = []
    image_sources = []
    
    for item in image_dataset_hf:
        image_data.append(item['image'])
        
        # Map to stress categories
        label = [0] * NUM_LABELS
        if 'label' in item:
            label_str = str(item['label']).lower()
            if any(kw in label_str for kw in ['disease', 'blight', 'rust', 'spot']):
                label[3] = 1
            else:
                label[np.random.randint(0, NUM_LABELS)] = 1
        else:
            label[3] = 1
        
        image_labels.append(label)
        image_sources.append('real_hf')
    
    print("\n📊 Image dataset info:")
    print(f"   Total images: {len(image_data)}")
    print(f"   All from real HuggingFace datasets")
    
else:
    print("\n⚠️ No real images loaded, using synthetic fallback...")
    image_data = []
    image_labels = []
    image_sources = []
    
    for i in range(2000):
        img = np.random.randint(50, 200, (224, 224, 3), dtype=np.uint8)
        img[:, :, 1] = np.clip(img[:, :, 1] + 50, 0, 255)
        image_data.append(Image.fromarray(img))
        
        label = [0] * NUM_LABELS
        label[np.random.randint(0, NUM_LABELS)] = 1
        image_labels.append(label)
        image_sources.append('synthetic')
    
    print(f"   Synthetic images: {len(image_data)}")

# Image label distribution
print("\n📊 Image label distribution:")
image_label_counts = np.zeros(NUM_LABELS)
for labels in image_labels:
    for i, val in enumerate(labels):
        if val == 1:
            image_label_counts[i] += 1
for i, count in enumerate(image_label_counts):
    print(f"   {ISSUE_LABELS[i]:15s}: {int(count):4d} samples")

print(f"\n✅ Total datasets loaded successfully")
print(f"   Text: {len(text_data)} samples from {len(set(text_sources))} sources")
print(f"   Images: {len(image_data)} samples")

## 🔀 Step 6: Create Non-IID Data Splits

In [ ]:
def create_non_iid_split(data, labels, num_clients, alpha=0.5):
    """Create non-IID data split using Dirichlet distribution."""
    print(f"\n🔀 Creating non-IID split (Dirichlet α={alpha})...")
    
    labels_array = np.array(labels)
    
    # Get primary label
    label_indices = []
    for label in labels_array:
        if isinstance(label, list):
            positive_labels = [i for i, v in enumerate(label) if v == 1]
        else:
            positive_labels = np.where(label == 1)[0].tolist()
        
        if positive_labels:
            label_indices.append(positive_labels[0])
        else:
            label_indices.append(0)
    label_indices = np.array(label_indices)
    
    client_indices = [[] for _ in range(num_clients)]
    
    for k in range(NUM_LABELS):
        idx_k = np.where(label_indices == k)[0]
        if len(idx_k) == 0:
            continue
        np.random.shuffle(idx_k)
        
        proportions = np.random.dirichlet(np.repeat(alpha, num_clients))
        proportions = np.cumsum(proportions)
        split_points = (proportions * len(idx_k)).astype(int)[:-1]
        
        for client_id, idx_subset in enumerate(np.split(idx_k, split_points)):
            client_indices[client_id].extend(idx_subset.tolist())
    
    for i in range(num_clients):
        np.random.shuffle(client_indices[i])
        print(f"   Client {i}: {len(client_indices[i])} samples")
    
    return client_indices

NUM_CLIENTS = 5
text_client_indices = create_non_iid_split(text_data, text_labels, NUM_CLIENTS, 0.5)
image_client_indices = create_non_iid_split(image_data, image_labels, NUM_CLIENTS, 0.5)

print("\n✅ Non-IID splits created")

## 🏗️ Step 7: Model Architectures (LLM, ViT, VLM)

In [ ]:
class MultiModalDataset(Dataset):
    def __init__(self, texts=None, images=None, labels=None, sources=None, 
                 tokenizer=None, image_transform=None, processor=None, max_length=128):
        self.texts = texts
        self.images = images
        self.labels = labels
        self.sources = sources
        self.tokenizer = tokenizer
        self.image_transform = image_transform
        self.processor = processor  # For CLIP/BLIP
        self.max_length = max_length
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {}
        
        # Text encoding
        if self.texts is not None and self.tokenizer is not None:
            text = str(self.texts[idx])
            encoded = self.tokenizer(
                text,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            item['input_ids'] = encoded['input_ids'].squeeze(0)
            item['attention_mask'] = encoded['attention_mask'].squeeze(0)
        
        # Image encoding
        if self.images is not None:
            img = self.images[idx]
            if isinstance(img, str):
                img = Image.open(img).convert('RGB')
            elif isinstance(img, np.ndarray):
                img = Image.fromarray(img)
            
            if self.processor is not None:
                # Use processor for CLIP/BLIP
                if self.texts is not None:
                    # VLM: both text and image
                    encoded = self.processor(
                        text=str(self.texts[idx]),
                        images=img,
                        return_tensors='pt',
                        padding='max_length',
                        max_length=self.max_length,
                        truncation=True
                    )
                    for k, v in encoded.items():
                        item[k] = v.squeeze(0)
                else:
                    # Image only
                    encoded = self.processor(images=img, return_tensors='pt')
                    item['pixel_values'] = encoded['pixel_values'].squeeze(0)
            elif self.image_transform is not None:
                # Standard ViT
                item['pixel_values'] = self.image_transform(img)
        
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        if self.sources is not None:
            item['source'] = self.sources[idx]
        
        return item

image_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✅ Dataset class defined")

In [ ]:
class FederatedLLM(nn.Module):
    """LLM models: T5, BERT, GPT-2 families (9 models)"""
    def __init__(self, model_name, num_labels, use_lora=False):
        super().__init__()
        self.model_name = model_name
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size
        
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_labels)
        )
        
        if use_lora and HAS_PEFT:
            target_modules = get_lora_target_modules(model_name)
            lora_config = LoraConfig(
                r=8,
                lora_alpha=16,
                target_modules=target_modules,
                lora_dropout=0.1,
                bias="none"
            )
            self.encoder = get_peft_model(self.encoder, lora_config)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
            pooled = outputs.pooler_output
        else:
            pooled = outputs.last_hidden_state[:, 0]
        return self.classifier(pooled)


class FederatedViT(nn.Module):
    """ViT models: ViT-Base, ViT-Large, DeiT (4 models)"""
    def __init__(self, model_name, num_labels, use_lora=False):
        super().__init__()
        self.model_name = model_name
        self.encoder = ViTModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size
        
        self.classifier = nn.Sequential(
            nn.LayerNorm(hidden_size),
            nn.Linear(hidden_size, 512),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_labels)
        )
        
        if use_lora and HAS_PEFT:
            target_modules = get_lora_target_modules(model_name)
            lora_config = LoraConfig(
                r=8,
                lora_alpha=16,
                target_modules=target_modules,
                lora_dropout=0.1,
                bias="none"
            )
            self.encoder = get_peft_model(self.encoder, lora_config)
    
    def forward(self, pixel_values):
        outputs = self.encoder(pixel_values=pixel_values)
        pooled = outputs.pooler_output if hasattr(outputs, 'pooler_output') else outputs.last_hidden_state[:, 0]
        return self.classifier(pooled)


class FederatedVLM(nn.Module):
    """VLM models: CLIP, BLIP (4 models)"""
    def __init__(self, model_name, num_labels, use_lora=False):
        super().__init__()
        self.model_name = model_name
        
        if "clip" in model_name.lower():
            self.encoder = CLIPModel.from_pretrained(model_name)
            hidden_size = self.encoder.config.projection_dim
        elif "blip2" in model_name.lower():
            self.encoder = Blip2ForConditionalGeneration.from_pretrained(model_name)
            hidden_size = self.encoder.config.text_config.hidden_size
        elif "blip" in model_name.lower():
            self.encoder = BlipForConditionalGeneration.from_pretrained(model_name)
            hidden_size = self.encoder.config.text_config.hidden_size
        else:
            raise ValueError(f"Unsupported VLM: {model_name}")
        
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_labels)
        )
        
        if use_lora and HAS_PEFT:
            target_modules = get_lora_target_modules(model_name)
            lora_config = LoraConfig(
                r=8,
                lora_alpha=16,
                target_modules=target_modules,
                lora_dropout=0.1,
                bias="none"
            )
            if "clip" in model_name.lower():
                self.encoder.vision_model = get_peft_model(self.encoder.vision_model, lora_config)
                self.encoder.text_model = get_peft_model(self.encoder.text_model, lora_config)
            else:
                self.encoder.vision_model = get_peft_model(self.encoder.vision_model, lora_config)
    
    def forward(self, input_ids=None, attention_mask=None, pixel_values=None):
        if "clip" in self.model_name.lower():
            outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                pixel_values=pixel_values,
                return_dict=True
            )
            # Combine text and image embeddings
            pooled = (outputs.text_embeds + outputs.image_embeds) / 2
        else:
            # BLIP models
            outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                pixel_values=pixel_values,
                return_dict=True
            )
            pooled = outputs.decoder_hidden_states[-1][:, 0] if hasattr(outputs, 'decoder_hidden_states') else outputs.last_hidden_state[:, 0]
        
        return self.classifier(pooled)

print("✅ All model architectures defined (LLM, ViT, VLM)")

## 🔥 Step 8: Training Functions with Communication Efficiency Tracking

In [ ]:
def train_one_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    criterion = nn.BCEWithLogitsLoss()
    
    for batch in dataloader:
        batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        labels = batch.pop('labels')
        batch.pop('source', None)
        
        logits = model(**batch)
        loss = criterion(logits, labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)


def evaluate_model_with_sources(model, dataloader, device):
    """Evaluate model and track performance by dataset source."""
    model.eval()
    all_preds = []
    all_labels = []
    all_sources = []
    total_loss = 0
    criterion = nn.BCEWithLogitsLoss()
    
    with torch.no_grad():
        for batch in dataloader:
            sources = batch.pop('source', None)
            batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
            labels = batch.pop('labels')
            
            logits = model(**batch)
            loss = criterion(logits, labels)
            total_loss += loss.item()
            
            preds = torch.sigmoid(logits).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
            
            if sources is not None:
                if isinstance(sources, list):
                    all_sources.extend(sources)
                else:
                    all_sources.append(sources)
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    preds_binary = (all_preds > 0.5).astype(int)
    
    metrics = {
        'loss': total_loss / len(dataloader),
        'f1_macro': f1_score(all_labels, preds_binary, average='macro', zero_division=0),
        'accuracy': accuracy_score(all_labels, preds_binary),
        'precision': precision_score(all_labels, preds_binary, average='macro', zero_division=0),
        'recall': recall_score(all_labels, preds_binary, average='macro', zero_division=0)
    }
    
    # Calculate per-source metrics
    if all_sources:
        source_metrics = {}
        unique_sources = set(all_sources)
        for source in unique_sources:
            source_mask = np.array([s == source for s in all_sources])
            if source_mask.sum() > 0:
                source_f1 = f1_score(
                    all_labels[source_mask],
                    preds_binary[source_mask],
                    average='macro',
                    zero_division=0
                )
                source_metrics[source] = {
                    'f1': source_f1,
                    'count': source_mask.sum()
                }
        metrics['by_source'] = source_metrics
    
    return metrics


def calculate_communication_cost(model):
    """Calculate total bytes transmitted in federated round."""
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    bytes_transmitted = total_params * 4  # 4 bytes per float32
    mb_transmitted = bytes_transmitted / (1024 ** 2)
    return {
        'total_params': total_params,
        'mb_per_round': mb_transmitted
    }


def fedavg_aggregate(global_model, client_models, client_weights):
    """FedAvg aggregation with communication tracking."""
    global_dict = global_model.state_dict()
    
    for key in global_dict.keys():
        global_dict[key] = torch.stack([
            client_models[i].state_dict()[key].float() * client_weights[i]
            for i in range(len(client_models))
        ], dim=0).sum(0)
    
    global_model.load_state_dict(global_dict)
    return global_model

print("✅ Training functions with communication tracking defined")

## 🚀 Step 9: Configure ALL 17 Models

In [ ]:
# ALL 17 MODELS - Full configuration

LLM_MODELS = [
    'google/flan-t5-small',      # 1
    'google/flan-t5-base',       # 2
    't5-small',                  # 3
    'gpt2',                      # 4
    'gpt2-medium',               # 5
    'distilgpt2',                # 6
    'roberta-base',              # 7
    'bert-base-uncased',         # 8
    'distilbert-base-uncased',   # 9
]

VIT_MODELS = [
    'google/vit-base-patch16-224',   # 10
    'google/vit-large-patch16-224',  # 11
    'google/vit-base-patch16-384',   # 12
    'facebook/deit-base-patch16-224', # 13
]

VLM_MODELS = [
    'openai/clip-vit-base-patch32',           # 14
    'openai/clip-vit-large-patch14',          # 15
    'Salesforce/blip-image-captioning-base',  # 16
    'Salesforce/blip2-opt-2.7b',              # 17
]

# For quick testing, reduce to 2-3 models per category
# Comment out these lines to train ALL 17 models
LLM_MODELS = LLM_MODELS[:2]  # Train first 2 LLMs
VIT_MODELS = VIT_MODELS[:1]  # Train first 1 ViT
VLM_MODELS = VLM_MODELS[:1]  # Train first 1 VLM

# Results storage
all_results = {
    'federated': {},
    'centralized': {},
    'communication': {},
    'dataset_comparison': {}
}

print("\n" + "="*70)
print("COMPLETE 17-MODEL BENCHMARK CONFIGURATION")
print("="*70)
print(f"\n📊 Models configured:")
print(f"   LLM models: {len(LLM_MODELS)}")
print(f"   ViT models: {len(VIT_MODELS)}")
print(f"   VLM models: {len(VLM_MODELS)}")
print(f"   Total: {len(LLM_MODELS) + len(VIT_MODELS) + len(VLM_MODELS)}")
print(f"\n📊 Datasets:")
print(f"   Text sources: {len(set(text_sources))} ({len(text_data)} samples)")
print(f"   Image sources: {len(set(image_sources))} ({len(image_data)} samples)")
print(f"\n⏱️ Estimated time:")
total_models = len(LLM_MODELS) + len(VIT_MODELS) + len(VLM_MODELS)
print(f"   ~15-20 min per model × {total_models} models × 2 modes = {total_models * 30}-{total_models * 40} minutes")
print(f"\n🎯 Analysis:")
print(f"   1. Federated vs Centralized")
print(f"   2. Communication efficiency")
print(f"   3. Dataset source comparison")
print(f"   4. Model type comparison (LLM vs ViT vs VLM)")
print(f"   5. Paper benchmark comparison")